# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fec38ba0b80>
├── 'a' --> tensor([[1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604]])
└── 'x' --> <FastTreeValue 0x7fec38ba0df0>
    └── 'c' --> tensor([[ 0.8670,  3.1198,  0.3209,  0.6659],
                        [-1.1088,  1.0296,  0.1948, -0.1333],
                        [-0.4380, -1.0501, -1.0194, -1.3315]])

In [4]:
t.a

tensor([[1.2299, 0.3318, 0.4610],
        [0.6330, 0.5011, 0.8604]])

In [5]:
%timeit t.a

72.5 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fec38ba0b80>
├── 'a' --> tensor([[-0.5965, -0.8603, -0.3423],
│                   [ 0.8016,  0.9625, -0.8350]])
└── 'x' --> <FastTreeValue 0x7fec38ba0df0>
    └── 'c' --> tensor([[ 0.8670,  3.1198,  0.3209,  0.6659],
                        [-1.1088,  1.0296,  0.1948, -0.1333],
                        [-0.4380, -1.0501, -1.0194, -1.3315]])

In [7]:
%timeit t.a = new_value

82.3 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604]]),
    x: Batch(
           c: tensor([[ 0.8670,  3.1198,  0.3209,  0.6659],
                      [-1.1088,  1.0296,  0.1948, -0.1333],
                      [-0.4380, -1.0501, -1.0194, -1.3315]]),
       ),
)

In [10]:
b.a

tensor([[1.2299, 0.3318, 0.4610],
        [0.6330, 0.5011, 0.8604]])

In [11]:
%timeit b.a

66.6 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.8299,  0.1515,  0.7221],
               [ 0.8213, -0.9031,  1.3855]]),
    x: Batch(
           c: tensor([[ 0.8670,  3.1198,  0.3209,  0.6659],
                      [-1.1088,  1.0296,  0.1948, -0.1333],
                      [-0.4380, -1.0501, -1.0194, -1.3315]]),
       ),
)

In [13]:
%timeit b.a = new_value

636 ns ± 5.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 33.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

289 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

271 µs ± 6.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7feb8d8ea880>
├── 'a' --> tensor([[[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]],
│           
│                   [[1.2299, 0.3318, 0.4610],
│                    [0.6330, 0.5011, 0.8604]]])
└── 'x' --> <FastTreeValue 0x7feb8d8ea160>
    └── 'c' --> tensor([[[ 0.8670,  3.1198,  0.3209,  0.6659],
           

In [21]:
%timeit t_stack(trees)

44.3 µs ± 350 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7feb8d8ea2e0>
├── 'a' --> tensor([[1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604],
│                   [1.2299, 0.3318, 0.4610],
│                   [0.6330, 0.5011, 0.8604]])
└── 'x' --> <FastTreeValue 0x7feb8d8ea640>
    └── 'c' --> tensor([[ 0.8670,  3.1198,  0.3209,  0.6659],
                        [-1.1088,  1.0296,  0.1948, -0.1333],
                        [-0.4380, -1.0501, -1.0194, -1.3315],
   

In [23]:
%timeit t_cat(trees)

41.3 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.9 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]],
       
               [[1.2299, 0.3318, 0.4610],
                [0.6330, 0.5011, 0.8604]]]),
    x: Batch(
           c: tensor([[[ 0.8670,  3.1198,  0.3209,  0.6659],
                       [-1.1088,  1.0296,  0.1948, -0.1333],
                       [-0.4380, -1.0501, -1.0194, -1.3315]],
              
                      [[ 0.8670,  3.1198, 

In [26]:
%timeit Batch.stack(batches)

104 µs ± 2.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604],
               [1.2299, 0.3318, 0.4610],
               [0.6330, 0.5011, 0.8604]]),
    x: Batch(
           c: tensor([[ 0.8670,  3.1198,  0.3209,  0.6659],
                      [-1.1088,  1.0296,  0.1948, -0.1333],
                      [-0.4380, -1.0501, -1.0194, -1.3315],
                      [ 0.8670,  3.1198,  0.3209,  0.6659],
                      [-1.1088,  1.0296,  0.1948, -0.1333],
                     

In [28]:
%timeit Batch.cat(batches)

196 µs ± 4.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

591 µs ± 21.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
